# Extract paragraphs, combine short paragraphs

In [9]:
import re
import os
import csv
from nltk.tokenize import sent_tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/yuchenluo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [16]:
col_names = ["text","candidate", "term", "title", "comp"]
rows = []
for root, dirs, files, in os.walk('/Users/yuchenluo/Desktop/Measure_RadicalDiscourse'):
    for file in files:
        if file.endswith('txt'):
            speech = open(os.path.join(root, file), "r").read()
            #speech = f.read()
            #get meta data of the speech
            cand = re.sub("-speech.*", "", file)
            term = re.search("speech-(.*)-", file).group(1)[0:4]
            title = re.search("speech-(.*).txt", file).group(1)
            pars = speech.split('\n')
            pars = [i for i in pars if i] #remove empty strings
            pars = [re.sub(" \[[^()]*\]", "", par) for par in pars] #remove strings inside brackets
            # loop through all paragraphs inside each speech 
            i = 0    
            while i < (len(pars) - 1):
                n_sent = len(sent_tokenize(pars[i]))
                if n_sent >2:
                    row = [pars[i], cand, term, title, False]
                    rows.append(row)
                    i += 1
                elif i < len(pars) - 1:
                    row = [pars[i] + " " + pars[i +1], cand, term, title, True]
                    i += 2
                    if len(sent_tokenize(row[0])) < 3 and i < len(pars) - 1:
                        row_2 = [row[0] + " " + pars[i], cand, term, title, True]
                        i += 1
                        rows.append(row_2)
                    else: 
                        rows.append(row)
                else: 
                    row = [pars[i - 1] + " " + pars[i], cand, term, title, True]
                    rows.pop()
                    rows.append(row)
                            
                                
                            
                
    

AttributeError: 'NoneType' object has no attribute 'group'

In [1]:
#rows[0:10]  #check data

In [12]:
with open("paragraphs.csv", 'w') as csvfile:
    writer = csv.writer(csvfile)   
    writer.writerow(col_names)
    writer.writerows(rows)

# Repeat the process for Annenberg data

In [7]:
import pandas as pd
annen_meta = pd.read_csv("/Users/yuchenluo/Desktop/Measure_RadicalDiscourse/annenberg_metadata.csv")

In [11]:
root = "/Users/yuchenluo/Desktop/Measure_RadicalDiscourse/Annenberg-data" 
col_names = ["Speech_id","text","party", "term", "comp"] 
rows = [] 

for file in os.listdir('/Users/yuchenluo/Desktop/Measure_RadicalDiscourse/Annenberg-data'): 
    if file.endswith('txt'): 
        speech = open(os.path.join(root, file), "r").read() 
        #get meta data of the speech 
        id_speech = re.search('File_(.*).txt', file).group(1) 
        party = annen_meta[annen_meta['id_speech'] == float(id_speech)]['party'].iloc[0] 
        term = annen_meta[annen_meta['id_speech'] == float(id_speech)]['year'].iloc[0] 
        pars = speech.split('\n\n') 
        # the last paragraph is not part of body text
        del pars[-1] 
        pars = [i for i in pars if i] #remove empty strings 
        pars = [re.sub(" \[[^()]*\]", "", par) for par in pars] #remove strings inside brackets 
        pars = [re.sub("\t", "",par) for par in pars]
        pars = [re.sub("\n", "",par) for par in pars] # remove \t and \n
        # loop through all paragraphs inside each speech  
        i = 0     
        while i < (len(pars) - 1): 
            n_sent = len(sent_tokenize(pars[i])) 
            if n_sent >2: 
                row = [id_speech, pars[i], party, term, False] 
                rows.append(row) 
                i += 1 
            elif i < len(pars) - 1: 
                row = [id_speech, pars[i] + " " + pars[i +1], party, term, True] 
                i += 2 
                if len(sent_tokenize(row[0])) < 3 and i < len(pars) - 1: 
                    row_2 = [id_speech, row[0] + " " + pars[i], party, term, True] 
                    i += 1 
                    rows.append(row_2) 
                else:  
                    rows.append(row) 
            else:  
                row = [id_speech, pars[i - 1] + " " + pars[i], party, term, True] 
                rows.pop() 
                rows.append(row) 

In [88]:
with open("paragraphs_annenberg.csv", 'w') as csvfile: 
    writer = csv.writer(csvfile)    
    writer.writerow(col_names) 
    writer.writerows(rows)

[' To Chairman Dean and my great friend Dick Durbin; and to all my fellow citizens of this great nation; With profound gratitude and great humility, I accept your nomination for the presidency of the United States.']

"\nWhen we start talking about the economy, it's best that we get right to the point. Jimmy Carter promised the American people he would give them an inflation rate of four percent and an unemployment rate of four percent. There's no nice way to say it: he just plain broke his promise and by doing so, has shattered the hopes of millions and millions of Americans who are strapped to the wall by this unstable economy."